# Notebook 05: Intégration et Test d'une Équipe CrewAI (Document Deep Dive Crew)

Ce notebook se concentre sur le test et la démonstration de notre équipe d'agents CrewAI spécialisée dans l'analyse approfondie de documents : la `DocumentAnalysisCrew`. Nous allons instancier cette équipe en utilisant le module `src.agents.crewai_teams.document_analysis_crew`, lui soumettre un exemple de document et un focus de recherche, et examiner le rapport d'analyse qu'elle produit.

**Prérequis :**
* Environnement configuré via `00_setup_environment.ipynb`.
* Les bibliothèques `crewai` et `langchain-openai` (ou un autre fournisseur LLM configuré) doivent être installées (via `environment.yml` ou `requirements.txt`).
* Une clé API OpenAI (`OPENAI_API_KEY`) doit être configurée dans votre fichier `.env`, car les agents CrewAI l'utiliseront par défaut via la fonction `get_llm` de notre projet.

In [ ]:
import logging
import sys
from pathlib import Path
import os
import json # Pour un affichage lisible

# Ajout de la racine du projet au PYTHONPATH
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"Ajout de {project_root} au PYTHONPATH")

from dotenv import load_dotenv
dotenv_path = project_root / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION: Fichier .env non trouvé à {dotenv_path}.")

from config.settings import settings
from config.logging_config import setup_logging

# Importer la fonction pour exécuter notre équipe CrewAI
from src.agents.crewai_teams.document_analysis_crew import run_document_deep_dive_crew

# Configurer le logging pour le notebook
LOG_LEVEL_NOTEBOOK = "INFO" # Mettre à DEBUG pour voir les logs détaillés de CrewAI
setup_logging(level=LOG_LEVEL_NOTEBOOK) 
logger = logging.getLogger("nb_05_crewai_integration")

# --- MODIFIÉ : Vérification des prérequis pour le LLM utilisé par CrewAI (via get_llm) ---
active_llm_provider_for_crew = settings.DEFAULT_LLM_MODEL_PROVIDER.lower()
logger.info(f"L'équipe CrewAI utilisera le fournisseur LLM configuré par défaut : '{active_llm_provider_for_crew}' (via get_llm).")

if active_llm_provider_for_crew == "openai":
    if not settings.OPENAI_API_KEY:
        logger.error("ERREUR : Le fournisseur LLM pour CrewAI est 'openai', mais OPENAI_API_KEY n'est pas configurée. L'exécution de l'équipe CrewAI échouera probablement.")
        print("ERREUR: Clé API OpenAI manquante. Veuillez la configurer dans votre fichier .env.")
elif active_llm_provider_for_crew == "huggingface_api":
    if not settings.HUGGINGFACE_API_KEY:
        logger.error("ERREUR : Le fournisseur LLM pour CrewAI est 'huggingface_api', mais HUGGINGFACE_API_KEY n'est pas configurée.")
        print("ERREUR: Clé API HuggingFace manquante. Veuillez la configurer dans votre fichier .env.")
elif active_llm_provider_for_crew == "ollama":
    if not settings.OLLAMA_BASE_URL:
        logger.error("ERREUR : Le fournisseur LLM pour CrewAI est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée.")
        print("ERREUR: URL de base Ollama manquante. Veuillez la configurer dans votre fichier .env.")
    if not settings.OLLAMA_GENERATIVE_MODEL_NAME: # Vérifier aussi le nom du modèle pour Ollama
        logger.error("ERREUR : Le fournisseur LLM pour CrewAI est 'ollama', mais OLLAMA_GENERATIVE_MODEL_NAME n'est pas configuré.")
        print("ERREUR: Nom du modèle génératif Ollama manquant. Veuillez le configurer dans votre fichier .env ou settings.py.")
# --- FIN MODIFIÉ ---

# Vérifier si crewai est installé
try:
    import crewai
    logger.info(f"Bibliothèque CrewAI version {crewai.__version__} importée.")
except ImportError:
    logger.error("La bibliothèque 'crewai' n'est PAS installée. Veuillez l'installer (pip install crewai) et relancer.")
    crewai = None # Pour que les cellules suivantes ne plantent pas si import échoue

### 1. Préparation des Données d'Exemple pour l'Analyse

Nous allons définir un exemple de contenu de document, son ID, et un focus de recherche spécifique pour tester notre `DocumentAnalysisCrew`.

In [ ]:
sample_doc_id = "arxiv_sample_2401.001"
sample_research_focus = "Identify the core methodology, key results, and explicitly stated limitations regarding sim-to-real transfer techniques."

sample_doc_content = """
Abstract: This research presents "TransferRL", a novel framework for enhancing sim-to-real transfer in robotic manipulation tasks. 
By employing adaptive instance normalization in the feature space of a vision-based policy and a curriculum of progressively complex simulated environments, 
TransferRL significantly reduces the reality gap. We demonstrate its efficacy on a pick-and-place task, achieving an 82% success rate on a physical KUKA LBR iiwa robot after training purely in simulation.

Introduction: Sim-to-real transfer is a pivotal challenge in applying deep reinforcement learning (RL) to real-world robotics. 
While training RL agents in simulation is often faster, safer, and more scalable than on physical systems, policies learned in simulation frequently
fail to generalize to the real world due to discrepancies in dynamics, visual appearance, and sensor noise. 
This paper addresses these challenges with a focus on visual domain adaptation.

Methodology: TransferRL consists of two main components. First, an image-to-image translation network based on CycleGAN is used to translate
simulated images to a more realistic style, though this is only used for initial policy pre-training. The core of TransferRL is an
encoder-decoder policy architecture where the encoder features are modulated by adaptive instance normalization (AdaIN) layers. 
These AdaIN layers are conditioned on a learned domain embedding, allowing the policy to adapt its visual processing dynamically.
The agent is trained using Soft Actor-Critic (SAC) on a curriculum of tasks in a custom OpenAI Gym environment built with PyBullet. 
The curriculum gradually increases object diversity and complexity of grasp poses. Domain randomization is applied to lighting, textures, and camera positions.

Results: On the primary pick-and-place task, TransferRL (82% success on physical robot) outperformed: 
(a) direct transfer without AdaIN or curriculum (35%), 
(b) AdaIN without curriculum (65%), and 
(c) curriculum without AdaIN (58%). 
The domain embedding was observed to cluster according to different simulation variations, suggesting successful domain identification. 
Qualitative analysis showed improved handling of novel object textures and lighting in the real world.

Limitations: The current implementation of TransferRL requires a small set of real-world images to learn the target domain embedding for AdaIN, making it not entirely "zero-shot" sim-to-real. 
The computational overhead of the AdaIN layers adds a minor latency to the policy execution. 
The system has only been tested on quasi-static pick-and-place tasks.

Conclusion: TransferRL offers an effective method for robust sim-to-real transfer in vision-based robotic manipulation by combining adaptive normalization with a structured training curriculum. Future work will explore fully zero-shot adaptation and application to dynamic tasks.
"""

logger.info(f"Prêt à analyser le document ID: {sample_doc_id}")
logger.info(f"Focus de la recherche: {sample_research_focus}")

### 2. Exécution de l'Équipe CrewAI `DocumentAnalysisCrew`

Nous appelons la fonction `run_document_deep_dive_crew` (définie dans `src/agents/crewai_teams/document_analysis_crew.py`) avec nos données d'exemple. Cette fonction orchestre l'équipe CrewAI pour produire un rapport d'analyse.

In [ ]:
final_report_from_crew = None

if crewai and settings.OPENAI_API_KEY:
    logger.info("Lancement de DocumentAnalysisCrew...")
    print("\n🔄 Lancement de l'équipe CrewAI pour l'analyse du document... Cela peut prendre quelques minutes, surtout avec verbose=2.\n")
    
    try:
        final_report_from_crew = run_document_deep_dive_crew(
            document_id=sample_doc_id,
            document_content=sample_doc_content,
            research_focus=sample_research_focus
        )
        
        print("\n\n--- 📝 Rapport d'Analyse Final Compilé par l'Équipe CrewAI ---")
        if final_report_from_crew:
            try:
                from IPython.display import display, Markdown
                display(Markdown(final_report_from_crew))
            except ImportError:
                print(final_report_from_crew)
        else:
            print("L'équipe CrewAI n'a pas retourné de rapport (vérifiez les logs).")

    except Exception as e:
        logger.error(f"Erreur lors de l'exécution de DocumentAnalysisCrew: {e}", exc_info=True)
        print(f"ERREUR lors de l'exécution de l'équipe CrewAI : {e}")
        final_report_from_crew = f"Erreur d'exécution : {e}"
else:
    if not crewai:
        print("Bibliothèque CrewAI non installée. Test sauté.")
    elif not settings.OPENAI_API_KEY: # Vérification redondante mais bonne pour la clarté ici
        print("Clé API OpenAI manquante. Test de l'équipe CrewAI sauté.")

### 3. Analyse du Rapport et Discussion

* Examinez attentivement le rapport généré par l'équipe CrewAI.
* **Structure :** Le `ReportCompilerAgent` a-t-il bien structuré le rapport en fonction des contributions des autres agents (Extracteur, Résumeur, Analyste Critique) ?
* **Contenu :**
    * Les informations clés extraites par l'`InfoExtractorAgent` (méthodologie, résultats, limitations) sont-elles précises et pertinentes par rapport au `research_focus` ?
    * Les résumés de sections (si l'`SectionSummarizerAgent` a pu les produire) sont-ils concis et informatifs ?
    * L'analyse critique du `CriticalAnalystAgent` (forces, faiblesses, contributions) est-elle judicieuse ?
* **Focus de Recherche :** Le rapport final est-il bien axé sur le `research_focus` que nous avons spécifié ?
* **Verbosité de CrewAI :** Si vous avez réglé `verbose=2` dans la définition de la `Crew` dans `document_analysis_crew.py`, vous avez dû voir le détail des pensées et actions de chaque agent. Cela permet de comprendre leur processus de "réflexion" et de collaboration.

Cette exécution isolée nous donne une idée de la capacité de l'équipe CrewAI à traiter une tâche d'analyse de document complexe et spécialisée.

### 4. Intégration Potentielle dans le Workflow LangGraph

Rappelons comment cette équipe CrewAI pourrait s'intégrer à notre système LangGraph principal :

1.  **Wrapper la CrewAI comme un Outil LangChain :**
    La fonction `run_document_deep_dive_crew` peut être transformée en un outil LangChain personnalisé à l'aide du décorateur `@tool`.
    ```python
    # Exemple conceptuel (à mettre dans tool_definitions.py si on l'intègre)
    # from langchain_core.tools import tool
    # from src.agents.crewai_teams.document_analysis_crew import run_document_deep_dive_crew

    # @tool
    # def deep_document_analysis_tool(document_id: str, document_content: str, research_focus: str) -> str:
    #     """
    #     Performs an in-depth analysis of a specific document using a specialized team of AI agents (CrewAI).
    #     Use this when a detailed, structured report on a single document is needed, focusing on specific aspects.
    #     Args:
    #         document_id (str): The unique identifier of the document.
    #         document_content (str): The full text content of the document to be analyzed.
    #         research_focus (str): Specific questions or areas the detailed analysis should concentrate on.
    #     Returns:
    #         str: A structured analytical report about the document.
    #     """
    #     return run_document_deep_dive_crew(document_id, document_content, research_focus)
    ```

2.  **Utilisation par un Agent LangGraph :**
    L'agent `DocumentAnalysisAgent` (défini dans `agent_architectures.py` pour LangGraph) pourrait se voir attribuer ce nouvel outil.
    Son prompt système serait mis à jour pour lui indiquer quand il est approprié de déléguer une analyse approfondie à cet outil plutôt que de simplement utiliser le `knowledge_base_retrieval_tool` pour des extraits.

3.  **Flux dans LangGraph :**
    * Le `ResearchPlannerAgent` identifie un document clé et un focus d'analyse.
    * Le routeur dirige vers le `DocumentAnalysisAgent` (LangGraph).
    * Cet agent, voyant la nécessité d'une analyse profonde, appelle le `deep_document_analysis_tool`.
    * Le `ToolNode` de LangGraph exécute cet outil (qui lance la CrewAI).
    * Le rapport de la CrewAI revient comme un `ToolMessage` dans l'état LangGraph.
    * L'`DocumentAnalysisAgent` (LangGraph) reçoit ce rapport et l'intègre potentiellement à d'autres analyses avant de passer la main au `SynthesisAgent`.

Cette approche hybride permet de combiner la flexibilité et la gestion d'état de LangGraph pour l'orchestration globale avec la collaboration structurée et orientée rôles de CrewAI pour des tâches spécifiques très délimitées.

## Conclusion

Ce notebook a démontré comment instancier et exécuter notre `DocumentAnalysisCrew` pour effectuer une analyse approfondie d'un document. Nous avons également discuté des pistes pour intégrer cette équipe CrewAI comme un composant spécialisé au sein de notre architecture LangGraph principale.

L'utilisation de CrewAI pour des sous-tâches complexes peut enrichir les capacités de notre "Cognitive Swarm".